In [ ]:
# ============================================================
# VISUALIZACIÓN EN COLAB: predictions.csv / predictions.geojson
# Mapa interactivo (Leaflet/Folium) + explicación didáctica
# ============================================================
#
# OBJETIVO
# --------
# Este bloque:
# 1) Monta Google Drive (porque ahí están los artefactos)
# 2) Carga predictions.csv (preferido) o predictions.geojson (fallback)
# 3) Limpia y valida lon/lat/prob_incendio
# 4) Dibuja un mapa interactivo en Colab con:
#    - Puntos coloreados por prob_incendio (0..1)
#    - HeatMap opcional (mejor si hay muchos puntos)
#    - Control de capas (encender/apagar)
#    - Leyenda (colormap) para entender colores
# 5) Guarda el mapa como HTML en Drive (para abrirlo fuera de Colab si quieres)
#
# ¿POR QUÉ FOLIUM?
# ---------------
# Folium crea mapas "Leaflet" dentro del notebook, con zoom, pan, tooltips...
# Es perfecto para Colab, sin necesidad de GIS pesado.
#
# NOTA SOBRE RENDIMIENTO (MUY IMPORTANTE)
# --------------------------------------
# Si tienes MUCHÍSIMOS puntos (p.ej. 200k, 1M...), dibujar cada punto como marcador
# puede ir lento o incluso colgar el navegador.
# Por eso incluimos:
# - "MAX_POINTS" para muestrear (sample) puntos
# - HeatMap para visualizar densidad y zonas calientes sin pintar todos los puntos
#
# EJEMPLO DIDÁCTICO:
# - Si tu dataset es una rejilla de 1000m (1 km), y tienes miles de celdas,
#   el mapa de puntos te deja "ver" dónde el modelo cree que el riesgo es alto
#   (colores más rojos/oscuros).
# - El HeatMap te da una visión "macro": manchas intensas donde se acumula prob alta.
#
# ============================================================

In [2]:
# ============================================================
# 0) Setup (instalar librerías + detectar Colab + reproducibilidad)
# ============================================================
# En Colab, algunas librerías vienen preinstaladas, pero no siempre.
# Folium y branca (para la leyenda) pueden no estar.
#
# Hacemos:
# - Detectar Colab
# - Montar Drive (si Colab)
# - Instalar folium si no está
# - Importar todo
#
# Nota:
# - No usamos seaborn.
# - Esto es solo visualización, no entrenamiento.
# ============================================================

import os
import json
import numpy as np
import pandas as pd

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

def is_colab() -> bool:
    """Devuelve True si estamos en Google Colab."""
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False

print("✅ [0] Entorno:", "Google Colab" if is_colab() else "No-Colab / local")
print("✅ [0] RANDOM_STATE:", RANDOM_STATE)

# --- Montar Drive (OBLIGATORIO si ARTIFACT_DIR está en Drive) ---
if is_colab():
    from google.colab import drive
    print("\n🔧 [0] Montando Google Drive...")
    drive.mount("/content/drive")
    print("✅ [0] Drive montado en /content/drive")
else:
    print("\n⚠️ [0] No estás en Colab.")
    print("   - Si tu ARTIFACT_DIR apunta a /content/drive/... puede no existir.")
    print("   - Ajusta ARTIFACT_DIR a una ruta válida en tu entorno.")

# --- Instalar folium si hace falta ---
try:
    import folium
    from folium.plugins import HeatMap
    print("✅ [0] folium disponible")
except Exception:
    print("📦 [0] folium no está instalado. Instalando...")
    if is_colab():
        !pip -q install folium
    else:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "folium"])
    import folium
    from folium.plugins import HeatMap
    print("✅ [0] folium instalado")

# --- Colormap (leyenda) ---
try:
    import branca.colormap as cm
    print("✅ [0] branca.colormap disponible")
except Exception:
    print("📦 [0] branca no está. Instalando...")
    if is_colab():
        !pip -q install branca
    else:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "branca"])
    import branca.colormap as cm
    print("✅ [0] branca instalado")

✅ [0] Entorno: Google Colab
✅ [0] RANDOM_STATE: 42

🔧 [0] Montando Google Drive...
Mounted at /content/drive
✅ [0] Drive montado en /content/drive
✅ [0] folium disponible
✅ [0] branca.colormap disponible


In [3]:
# ============================================================
# 1) Configuración de rutas (artefactos y ficheros de predicción)
# ============================================================
# Tu pipeline guardaba artefactos aquí:
#   ARTIFACT_DIR = "/content/drive/MyDrive/Piloto-Incendios/03-Artefactos"
#
# Y generaba:
# - predictions.csv
# - predictions.geojson
#
# Este script:
# - Intenta cargar predictions.csv (preferido)
# - Si no existe, intenta predictions.geojson
# ============================================================

ARTIFACT_DIR = "/content/drive/MyDrive/Piloto-Incendios/03-Artefactos"

PRED_CSV_PATH = os.path.join(ARTIFACT_DIR, "predictions.csv")
PRED_GEOJSON_PATH = os.path.join(ARTIFACT_DIR, "predictions.geojson")

print("\n📁 [1] ARTIFACT_DIR:", ARTIFACT_DIR)

if not os.path.isdir(ARTIFACT_DIR):
    raise FileNotFoundError(
        "No encuentro ARTIFACT_DIR.\n"
        "Checklist:\n"
        "- ¿Has montado Drive?\n"
        "- ¿Es correcta la ruta de ARTIFACT_DIR?\n"
        "- ¿Existe esa carpeta en tu Drive?"
    )

print("📌 [1] Buscando ficheros:")
print("   - predictions.csv    :", "✅" if os.path.isfile(PRED_CSV_PATH) else "❌", PRED_CSV_PATH)
print("   - predictions.geojson:", "✅" if os.path.isfile(PRED_GEOJSON_PATH) else "❌", PRED_GEOJSON_PATH)

if (not os.path.isfile(PRED_CSV_PATH)) and (not os.path.isfile(PRED_GEOJSON_PATH)):
    print("\n🧾 [1] Contenido actual de ARTIFACT_DIR:")
    print(os.listdir(ARTIFACT_DIR))
    raise FileNotFoundError(
        "No encuentro predictions.csv ni predictions.geojson.\n"
        "Ejecuta primero el pipeline de ML que los genera, o revisa rutas."
    )




📁 [1] ARTIFACT_DIR: /content/drive/MyDrive/Piloto-Incendios/03-Artefactos
📌 [1] Buscando ficheros:
   - predictions.csv    : ✅ /content/drive/MyDrive/Piloto-Incendios/03-Artefactos/predictions.csv
   - predictions.geojson: ✅ /content/drive/MyDrive/Piloto-Incendios/03-Artefactos/predictions.geojson


In [5]:
# ============================================================
# 2) Cargar predicciones (CSV preferido, GeoJSON como fallback)
# ============================================================
# ¿Por qué preferimos CSV?
# - Es más simple: columnas lon, lat, prob_incendio
# - pandas lo maneja perfecto
#
# ¿Cuándo usar GeoJSON?
# - Si quieres preservar geometrías como 'FeatureCollection'
# - Si el CSV no está
#
# Formato esperado mínimo:
# - lon: longitud (float)   ej: -6.7
# - lat: latitud (float)    ej: 42.6
# - prob_incendio: 0..1     ej: 0.73
#
# Columnas opcionales útiles:
# - pred_clase (0/1)
# - fire_label (0/1) si quieres comparar con la realidad
# ============================================================

pred = None
source_used = None

if os.path.isfile(PRED_CSV_PATH):
    print("\n📥 [2] Cargando predictions.csv ...")
    pred = pd.read_csv(PRED_CSV_PATH)
    source_used = "csv"
    print("✅ [2] CSV cargado:", pred.shape)

elif os.path.isfile(PRED_GEOJSON_PATH):
    print("\n📥 [2] Cargando predictions.geojson ...")
    with open(PRED_GEOJSON_PATH, "r", encoding="utf-8") as f:
        gj = json.load(f)

    rows = []
    for feat in gj.get("features", []):
        coords = feat.get("geometry", {}).get("coordinates", [None, None])
        props = feat.get("properties", {})
        rows.append({
            "lon": coords[0],
            "lat": coords[1],
            "prob_incendio": props.get("prob_incendio", np.nan),
            "pred_clase": props.get("pred_clase", np.nan),
            "fire_label": props.get("fire_label", np.nan),
        })

    pred = pd.DataFrame(rows)
    source_used = "geojson"
    print("✅ [2] GeoJSON cargado a DataFrame:", pred.shape)

else:
    raise RuntimeError("Inesperado: no se pudo cargar ni CSV ni GeoJSON.")

print("✅ [2] Fuente usada:", source_used)
print("\n🧾 [2] Vista rápida (head):")
display(pred.head(5))

required_cols = ["lon", "lat", "prob_incendio"]
missing = [c for c in required_cols if c not in pred.columns]
if missing:
    raise KeyError(
        f"Faltan columnas obligatorias: {missing}\n"
        f"Columnas disponibles: {list(pred.columns)}"
    )



📥 [2] Cargando predictions.csv ...
✅ [2] CSV cargado: (76179, 5)
✅ [2] Fuente usada: csv

🧾 [2] Vista rápida (head):


,lon,lat,fire_label,prob_incendio,pred_clase
0,-6.949592,42.300544,0,0.809311,1
1,-6.947346,42.300544,0,0.291073,0
2,-6.945100,42.300544,0,0.228057,0
3,-6.942854,42.300544,0,0.559029,1
4,-6.940608,42.300544,0,0.141363,0


In [6]:
# ============================================================
# 3) Limpieza y QA mínima (lon/lat/prob)
# ============================================================
# - A veces lon/lat vienen como texto (strings) -> hay que convertir a float
# - Puede haber filas sin coordenadas (NaN) si falló el parseo de .geo
# - prob_incendio debe estar en [0,1] (si no, algo raro pasó)
#
# Mini-ejemplo didáctico:
# - Si prob_incendio = 7.3 -> mal (probabilidad no puede ser > 1)
# - Si prob_incendio = -0.2 -> mal (<0)
# - Si lon/lat están vacíos -> no se puede dibujar en el mapa
# ============================================================

print("\n🧹 [3] Convirtiendo lon/lat/prob_incendio a numérico...")

pred["lon"] = pd.to_numeric(pred["lon"], errors="coerce")
pred["lat"] = pd.to_numeric(pred["lat"], errors="coerce")
pred["prob_incendio"] = pd.to_numeric(pred["prob_incendio"], errors="coerce")

# Filtramos filas válidas (con coordenadas y probabilidad)
pred_clean = pred.dropna(subset=["lon", "lat", "prob_incendio"]).copy()

print("✅ [3] Filas totales:", len(pred))
print("✅ [3] Filas válidas (lon/lat/prob):", len(pred_clean))

if len(pred_clean) == 0:
    raise ValueError("No hay filas válidas con lon/lat/prob_incendio. Revisa el export o el parseo de .geo.")

# Chequeo de rango de probabilidad (avisos, no crash)
out_low = (pred_clean["prob_incendio"] < 0).sum()
out_high = (pred_clean["prob_incendio"] > 1).sum()

print("\n🔎 [3] QA de prob_incendio (debe estar en [0,1]):")
if out_low == 0 and out_high == 0:
    print("✅ [3] OK: todas las probabilidades están dentro de [0,1]")
else:
    print(f"⚠️ [3] Hay valores fuera de [0,1]: {out_low} por debajo, {out_high} por encima.")
    print("   Esto suele indicar un error de cálculo o que no es una probabilidad real.")
    print("   Aun así, recortaremos (clip) para visualización.")
    pred_clean["prob_incendio"] = pred_clean["prob_incendio"].clip(0, 1)

# Chequeo rápido de rango lon/lat (España aprox: lon -10..5, lat 35..45)
# (Esto NO es una validación universal, solo un aviso útil)
lon_weird = ((pred_clean["lon"] < -30) | (pred_clean["lon"] > 30)).sum()
lat_weird = ((pred_clean["lat"] < 0) | (pred_clean["lat"] > 70)).sum()
print("\n🔎 [3] QA de coordenadas (avisos):")
if lon_weird == 0 and lat_weird == 0:
    print("✅ [3] Coordenadas parecen razonables (a ojo).")
else:
    print(f"⚠️ [3] Coordenadas raras detectadas: lon_weird={lon_weird}, lat_weird={lat_weird}")
    print("   Si lon/lat están intercambiadas o en otra proyección, el mapa saldrá mal.")

print("\n📌 [3] Estadísticos rápidos de prob_incendio:")
display(pred_clean["prob_incendio"].describe())


🧹 [3] Convirtiendo lon/lat/prob_incendio a numérico...
✅ [3] Filas totales: 76179
✅ [3] Filas válidas (lon/lat/prob): 76179

🔎 [3] QA de prob_incendio (debe estar en [0,1]):
✅ [3] OK: todas las probabilidades están dentro de [0,1]

🔎 [3] QA de coordenadas (avisos):
✅ [3] Coordenadas parecen razonables (a ojo).

📌 [3] Estadísticos rápidos de prob_incendio:


,prob_incendio
count,7.617900e+04
mean,1.841714e-01
std,2.622578e-01
min,3.611783e-16
25%,7.897208e-03
50%,4.340677e-02
75%,2.703127e-01
max,1.000000e+00


In [7]:
# ============================================================
# 4) Rendimiento: puntos vs muestreo vs HeatMap
# ============================================================
# DIBUJAR PUNTOS (CircleMarker)
# - Pros: puedes inspeccionar cada punto con tooltip.
# - Contras: muchos puntos = mapa lento.
#
# HEATMAP
# - Pros: muy visual para grandes volúmenes; más ligero que miles de marcadores.
# - Contras: no ves punto a punto, ves densidad/“mancha”.
#
# Estrategia recomendada:
# - Si N <= ~10k: dibuja puntos sin problema.
# - Si N > ~10k: muestrea para puntos + HeatMap para visión general.
# ============================================================

N_TOTAL = len(pred_clean)

MAX_POINTS = 20000      # si hay más, muestreamos
USE_HEATMAP = True      # HeatMap recomendado
HEATMAP_SHOW_BY_DEFAULT = False  # la capa HeatMap aparece apagada inicialmente

print("\n⚙️ [4] Configuración visual:")
print("   - N_TOTAL:", f"{N_TOTAL:,}")
print("   - MAX_POINTS:", f"{MAX_POINTS:,}")
print("   - USE_HEATMAP:", USE_HEATMAP)

if N_TOTAL > MAX_POINTS:
    print(f"⚠️ [4] Hay {N_TOTAL:,} puntos. Muestreamos a {MAX_POINTS:,} para marcadores.")
    pred_plot = pred_clean.sample(MAX_POINTS, random_state=RANDOM_STATE).copy()
else:
    pred_plot = pred_clean.copy()

print("✅ [4] Puntos para dibujar como marcadores:", f"{len(pred_plot):,}")


⚙️ [4] Configuración visual:
   - N_TOTAL: 76,179
   - MAX_POINTS: 20,000
   - USE_HEATMAP: True
⚠️ [4] Hay 76,179 puntos. Muestreamos a 20,000 para marcadores.
✅ [4] Puntos para dibujar como marcadores: 20,000


In [8]:
# ============================================================
# 5) Crear el mapa (centro + capas + leyenda)
# ============================================================
# ¿Cómo centramos el mapa?
# - Tomamos la media de lat/lon de los puntos dibujados.
# - No es perfecto, pero es un buen “centro” inicial.
#
# Leyenda de color:
# - Usamos un colormap de 0..1
# - Colores tipo amarillo->rojo (bajo->alto)
#
# IMPORTANTE:
# - Los colores son para “visualizar gradientes”, no para tomar decisiones finales.
# - El valor real útil está en prob_incendio y en cómo calibras el umbral.
# ============================================================

center_lat = float(pred_plot["lat"].mean())
center_lon = float(pred_plot["lon"].mean())

print("\n🗺️ [5] Centro del mapa (aprox):", (center_lat, center_lon))

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles="OpenStreetMap"
)

# Colormap (leyenda)
colormap = cm.linear.YlOrRd_09.scale(0, 1)
colormap.caption = "prob_incendio (0=bajo, 1=alto)"
colormap.add_to(m)

print("✅ [5] Mapa base creado + leyenda añadida")


🗺️ [5] Centro del mapa (aprox): (42.52506359188306, -6.526932815829078)
✅ [5] Mapa base creado + leyenda añadida


In [9]:
# ============================================================
# 6) Capa de puntos (CircleMarker) coloreados por prob_incendio
# ============================================================
# Cada punto:
# - se pinta en su lat/lon
# - su color depende de prob_incendio
# - tooltip muestra info al pasar el ratón
#
# Tooltip didáctico:
# - prob_incendio: probabilidad del modelo
# - pred_clase (si existe): 0/1 según umbral usado en pipeline
# - fire_label (si existe): etiqueta real (para comparar)
# ============================================================

points_layer = folium.FeatureGroup(name="Puntos (prob_incendio)", show=True)

has_pred_clase = "pred_clase" in pred_plot.columns
has_fire_label = "fire_label" in pred_plot.columns

print("\n🎯 [6] Añadiendo marcadores (esto puede tardar si hay muchos puntos)...")

# Ajusta estos parámetros visuales
POINT_RADIUS = 3
POINT_OPACITY = 0.80

for _, row in pred_plot.iterrows():
    p = float(row["prob_incendio"])
    color = colormap(p)

    tooltip_parts = [f"prob_incendio: {p:.3f}"]
    if has_pred_clase:
        try:
            tooltip_parts.append(f"pred_clase: {int(row['pred_clase'])}")
        except Exception:
            tooltip_parts.append(f"pred_clase: {row['pred_clase']}")
    if has_fire_label:
        try:
            tooltip_parts.append(f"fire_label: {int(row['fire_label'])}")
        except Exception:
            tooltip_parts.append(f"fire_label: {row['fire_label']}")

    tooltip = " | ".join(tooltip_parts)

    folium.CircleMarker(
        location=[float(row["lat"]), float(row["lon"])],
        radius=POINT_RADIUS,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=POINT_OPACITY,
        weight=0.5,
        tooltip=tooltip
    ).add_to(points_layer)

points_layer.add_to(m)
print("✅ [6] Capa de puntos añadida")


🎯 [6] Añadiendo marcadores (esto puede tardar si hay muchos puntos)...
✅ [6] Capa de puntos añadida


In [10]:
# ============================================================
# 7) (Opcional recomendado) HeatMap ponderado por prob_incendio
# ============================================================
# HeatMap usa triples: [lat, lon, weight]
# Aquí weight = prob_incendio
#
# Interpretación:
# - Zonas con muchos puntos y prob alta -> más “intensas” (más calientes)
# - Si hay puntos dispersos, la mancha será suave
#
# Parámetros:
# - radius: cuánto “se expande” la influencia de cada punto
# - blur: suavizado (más blur = más difuso)
# ============================================================

if USE_HEATMAP:
    print("\n🔥 [7] Añadiendo HeatMap (recomendado para visión global)...")
    heat_layer = folium.FeatureGroup(name="HeatMap (prob_incendio)", show=HEATMAP_SHOW_BY_DEFAULT)

    heat_data = pred_plot[["lat", "lon", "prob_incendio"]].values.tolist()
    HeatMap(
        heat_data,
        radius=12,
        blur=18,
        max_zoom=13
    ).add_to(heat_layer)

    heat_layer.add_to(m)
    print("✅ [7] HeatMap añadido (controlable desde capas)")


🔥 [7] Añadiendo HeatMap (recomendado para visión global)...
✅ [7] HeatMap añadido (controlable desde capas)


In [11]:
# ============================================================
# 8) Control de capas + guardar HTML + mostrar en Colab
# ============================================================
# Control de capas:
# - Te permite activar/desactivar Puntos y HeatMap
#
# Guardar HTML:
# - Te deja abrir el mapa fuera de Colab (en tu navegador)
# - Muy útil para compartir o archivar
#
# Mostrar en Colab:
# - Si la última línea de la celda es "m", Colab renderiza el mapa.
# ============================================================

folium.LayerControl(collapsed=False).add_to(m)

MAP_HTML_PATH = os.path.join(ARTIFACT_DIR, "mapa_predicciones.html")
m.save(MAP_HTML_PATH)

print("\n💾 [8] Mapa guardado como HTML en:", MAP_HTML_PATH)
print("👉 [8] En Colab verás el mapa debajo.")
print("    - Usa el control de capas (esquina superior derecha) para activar HeatMap.")
print("    - Pasa el ratón sobre puntos para ver tooltips con prob_incendio.")

m


Output hidden; open in https://colab.research.google.com to view.